In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Optional
from langchain_community.callbacks import get_openai_callback

## Upload Code

In [2]:
import os
file_path = r"D:\project-ruang-guru\Debugging\Code\Source-Code-Analysis-using-GenAI\research\test_repo\rg-llm\llm\api\error_handlers.py"

with open(file_path, 'r') as rp:
    code = rp.read()
    
code = "Sources : " + file_path +"\n\n" +"**Code **: \n"+ code

In [3]:
print(code)

Sources : D:\project-ruang-guru\Debugging\Code\Source-Code-Analysis-using-GenAI\research\test_repo\rg-llm\llm\api\error_handlers.py

**Code **: 
from fastapi import Request
from fastapi.applications import FastAPI
from fastapi.exceptions import RequestValidationError
from starlette.exceptions import HTTPException as StarletteHTTPException
from llm.core.service.exceptions import PlatformModelNotFoundError, AIException
from llm.api.views import error_response
from typing import Union

import logging


logger = logging.getLogger(__name__)


async def request_validation_handler(_: Request, error: RequestValidationError):
    validation_errors = [(".".join(str(x) for x in e["loc"]), e["type"])
                         for e in error.errors()]
    return error_response(error, 400, validation_errors)


async def http_exception_handler(_, error: StarletteHTTPException):
    return error_response(error)


async def platform_model_exception_handler(request: Request, error: PlatformModelNotFoundE

## Define the LLM Model

In [4]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2)

## Langgraph Model

### Agent Localizer

In [5]:
class Localizer(BaseModel):
    predicted_bug: str = Field(
        description="A description of the predicted bug in the code."
    )
    
def localizer_invoke(requirement, code):    
    localization_gen_prompt = ChatPromptTemplate.from_template(
    '''
    **Role**: As a localizer, your task is to review the following code snippet and identify any potential bugs. Ignore any issues related to missing dependencies or packages. Focus on syntax errors, logical errors, and potential runtime issues within the given code. Provide a detailed explanation of each identified bug and suggest possible fixes. If no issues are found, simply state that the code is bug-free. If there are issues, explain them clearly.

    **Requirement**:
    {requirement}

    **Provided resources**:
    {code}

    The first row is counted after **Code**
    
    So it is like : 
    Source : path/to/file
    
    **Code** : 
    Row 1
    Row 2
    Row 3 
    And so on
    
    Your output must be in the following format:

    
    ```
    Predicted error code 1 -- source : path/to/file -- Row -- Explanation of the error and why it occurs
    Predicted error code 2 -- source : path/to/file -- Row -- Explanation of the error and why it occurs
    ```

    Please review the code accordingly.
    '''    
    )
    localizer_agent = create_structured_output_runnable(
        Localizer, llm, localization_gen_prompt
    ) 
    with get_openai_callback() as localizer_info:
        localizer_invoke = localizer_agent.invoke({'code':code,'requirement':requirement})
    return localizer_invoke, localizer_info
requirement = "Identify the possible bugs"
code = code
localizer_invoke, localizer_info = localizer_invoke(requirement, code)

c:\Users\Acer\anaconda3\envs\CodeDebugger\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: LangChain has introduced a method called `with_structured_output` that is available on ChatModels capable of tool calling. You can read more about the method here: <https://python.langchain.com/docs/modules/model_io/chat/structured_output/>.Please follow our extraction use case documentation for more guidelines on how to do information extraction with LLMs. <https://python.langchain.com/docs/use_cases/extraction/>. If you notice other issues, please provide feedback here: <https://github.com/langchain-ai/langchain/discussions/18154>
  warn_deprecated(


In [6]:
print(f"Localizer Invoke : \n\n{localizer_invoke.predicted_bug}\n\nLocalizer Info:\n\n{localizer_info}")

Localizer Invoke : 

The code is bug-free.

Localizer Info:

Tokens Used: 662
	Prompt Tokens: 649
	Completion Tokens: 13
Successful Requests: 1
Total Cost (USD): $0.000344


In [7]:
from pprint import pprint
pprint(localizer_invoke)

Localizer(predicted_bug='The code is bug-free.')


### Agent Explanation

In [8]:
class Explanation(BaseModel):
    function_explanation: str = Field(
        description="The explanation of the code."
    )

def explanation_invoke(code):
    expalantion_gen_prompt = ChatPromptTemplate.from_template(
    """ 
    **Role** : As an identifier, you will provide an explanation of the code given. Then, You should specify the workflow or processing steps within the code. 
    
    **Given code** : 
    {code}
    
    Analyze the code to identify the function and process on the code.
    
    Follow this structured format:

    1. **Class 1**: Provide a summarized overview of Class 1.
        - a) **Method or Async Method 1 of Class 1**: Describe the function and purpose of Method 1.
        - b) **Method Async Method 2 of Class 1**: Describe the function and purpose of Method 2.
        - Continue this pattern for all methods in Class 1.
    2. **Class 2**: Provide a summarized overview of Class 2.
        - a) **Method or Async Method 1 of Class 2**: Describe the function and purpose of Method 1.
        - b) **Method or Async Method 2 of Class 2**: Describe the function and purpose of Method 2.
        - Continue this pattern for all methods in Class 2.
        
    - Continue this pattern for all remaining classes.

    After detailing the functions of each class and method, provide an overall explanation of the code's workflow. 

    Explain the process of the given code, predicting the overall function. If the code involves any dependencies or packages that are not explicitly stated, please predict their function as well.
    """    
    )
    
    explanation_agent = create_structured_output_runnable(
        Explanation, llm, expalantion_gen_prompt
    )
    with get_openai_callback() as expalanation_info:
        explanation_invoke = explanation_agent.invoke({'code':code})
    return explanation_invoke, expalanation_info
code = code
explanation_invoke, explanation_info = explanation_invoke(code)

In [9]:
print(f"Explanation Invoke : \n\n{explanation_invoke.function_explanation}\n\nIdentifier Info:\n\n{explanation_info}")

Explanation Invoke : 

Sure! Let's analyze the provided code to identify the functions and processes within it.

Identifier Info:

Tokens Used: 737
	Prompt Tokens: 712
	Completion Tokens: 25
Successful Requests: 1
Total Cost (USD): $0.00039349999999999997


### Agent Repairer

In [10]:
class Repairer(BaseModel):
    repair_description: str = Field(
        description="A description of the code that needs to be repaired."
    )

def repairer_invoke(code, explanation, bug):        
    repairer_gen_prompt = ChatPromptTemplate.from_template(
    """
    **Role** : As a code repairer, your task is to provide the corrected version based on any predicted bugs and the explanation of the code. 

    You will be given the predicted bug(s). 
    
    **Code** : 
    {code}
    
    **The explanation of the code**
    {explanation}
    
    **Predicted bug** 
    {bug}

    For each bug, follow this format:
    
    Error 1:
    the code with error 1 -- specify the row(s)  
    
    After repair 1:
    `the code after repairing error 1`  
    Explanation: Explain what you have repaired.
    \n\n
    Error 2:
    the code with error 2 -- specify the row(s)  
    After repair 2:
    `the code after repairing error 2`  
    Explanation: Explain what you have repaired.
    \n\n
    Continue this pattern for any additional errors.

    """
    )
    repairer_agent = create_structured_output_runnable(
        Repairer, llm, repairer_gen_prompt
    )  
    with get_openai_callback() as repairer_info:
        repairer_invoke= repairer_agent.invoke({"code":code, "explanation":explanation, "bug":bug})
    return repairer_invoke, repairer_info

code = code
explanation = explanation_invoke.function_explanation
bug = localizer_invoke.predicted_bug

repairer_invoke, repairer_info = repairer_invoke(code, explanation, bug)

In [11]:
print(f"Repairer Invoke : \n\n{repairer_invoke.repair_description}\n\nRepairer Info:\n\n{repairer_info}")

Repairer Invoke : 

The code is bug-free.

Repairer Info:

Tokens Used: 634
	Prompt Tokens: 621
	Completion Tokens: 13
Successful Requests: 1
Total Cost (USD): $0.00033


### Agent Crafter

In [14]:
class Crafter(BaseModel):
    code: str = Field(
        description="The code after reviewed"
    )
    
def crafter_invoke(code, repaired_description):
    crafter_gen_prompt = ChatPromptTemplate.from_template(
    """
    **Role**: As a crafter, your task is to merge the corrected code into the original code if it should be corrected.
    **The given code:**
    {code}
    
    The first row is counted after **Code**
    
    So it is like : 
    Source : path/to/file
    
    **Code** : 
    Row 1
    Row 2
    Row 3 
    And so on

    **The description of the code that should be repaired:**
    {repaired_description}
    """
    )
    crafter_agent = create_structured_output_runnable(
        Crafter, llm, crafter_gen_prompt
    )
    with get_openai_callback() as crafter_info:
        crafter_invoke=crafter_agent.invoke({'code':code,'repaired_description':repaired_description})   
    
    return crafter_invoke,crafter_info

code = code
repaired_description = repairer_invoke.repair_description
crafter_invoke,crafter_info = crafter_invoke(code, repaired_description)

In [15]:
print(f"Crafter Invoke : \n\n{crafter_invoke.code}\n\nCrafter Info:\n\n{crafter_info}")

Crafter Invoke : 

from fastapi import Request
from fastapi.applications import FastAPI
from fastapi.exceptions import RequestValidationError
from starlette.exceptions import HTTPException as StarletteHTTPException
from llm.core.service.exceptions import PlatformModelNotFoundError, AIException
from llm.api.views import error_response
from typing import Union

import logging


logger = logging.getLogger(__name__)


async def request_validation_handler(_: Request, error: RequestValidationError):
    validation_errors = [(".".join(str(x) for x in e["loc"]), e["type"])
                         for e in error.errors()]
    return error_response(error, 400, validation_errors)


async def http_exception_handler(_, error: StarletteHTTPException):
    return error_response(error)


async def platform_model_exception_handler(request: Request, error: PlatformModelNotFoundError):
    logger.error(f"Not Found for {error.field}: {error.value}")
    return error_response(error, 400)


async def ai_ex

In [ ]:
class Developer(BaseModel):
    refactored_code: str = Field(
        description="The refactored and optimized Python code."
    )

def developer_invoke(repaired_code : str, lang:str):
    developer_gen_prompt = ChatPromptTemplate.from_template(
    """
    Your task as an expert developer,  you are given the code that needs to be refactored and optimized for better performance, readability, and maintainability. Below are the details and the code snippet. Please refactor the code, provide comments where necessary, and suggest any improvements or best practices that can be applied.

    Details:

    - The code should follow {lang} best practices.
    - Ignoring the missing depedencies or packages
    - Aim for improved performance without sacrificing readability.
    - Ensure that the code is modular and easy to maintain.
    - Add comments to explain complex sections of the code.
    - Optimize any redundant or inefficient parts of the code.
    - Ensure proper error handling and input validation.
    - Follow naming conventions for variables and functions.

    Code Snippet:
    {code}

    Please provide the refactored and optimized version of the code along with any comment of explanations or suggestions for improvements."""
    )
    developer_agent = create_structured_output_runnable(
        Developer, llm, developer_gen_prompt
    )
    with get_openai_callback() as developer_info:
        developer_invoke=developer_agent.invoke({"lang":lang,"code":repaired_code})
    return developer_invoke,developer_info

repaired_code = crafter_invoke.improved_code
lang = "Python"
developer_invoke,developer_info = developer_invoke(repaired_code, lang)


In [ ]:
print(f"Developer Invoke : \n\n{developer_invoke.refactored_code}\n\Developer Info:\n\n{developer_info}")

Developer Invoke : 

from fastapi import Request, FastAPI
from fastapi.exceptions import RequestValidationError
from starlette.exceptions import HTTPException as StarletteHTTPException
from llm.core.service.exceptions import PlatformModelNotFoundError, AIException
from llm.api.views import error_response
from typing import Union
import logging

# Initialize logger
logger = logging.getLogger(__name__)

# Handler for request validation errors
async def request_validation_handler(_: Request, error: RequestValidationError):
    validation_errors = [
        (".".join(str(x) for x in e.get("loc", [])), e.get("type", "unknown"))
        for e in error.errors()
    ]
    return error_response(error, 400, validation_errors)

# Handler for HTTP exceptions
async def http_exception_handler(_: Request, error: StarletteHTTPException):
    return error_response(error, 500)

# Handler for platform model not found errors
async def platform_model_exception_handler(_: Request, error: PlatformModelNotFou